In [49]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [121]:
%sendtofile --source "../Sensor_Kennel/BME280_funcs.py"

Sent 77 lines (3294 bytes) to BME280_funcs.py.


In [122]:
from machine import I2C, Pin
import time, os, sys, ustruct

#i2c = I2C(1)  # on Pi
i2c = I2C(1, scl=Pin(25), sda=Pin(27)) # on ESP32 pin26 broken


print(list(map(hex, i2c.scan())))  # 0x23=LTR-559, 0x76=BME280, 0x49=ADS1015

['0x23', '0x49', '0x76']


In [123]:
import BME280_funcs
BME280_funcs.bme280init(i2c, 0x76)
for i in range(20):
    x = BME280_funcs.readBME280(0x76)
    print("temp, baro, humid", x)
    time.sleep_ms(500)

temp, baro, humid (22.66, 64428.22, 86.18164)
temp, baro, humid (19.91, 101514.9, 39.11035)
temp, baro, humid (19.94, 101515.6, 39.13672)
temp, baro, humid (19.96, 101519.0, 39.11524)
temp, baro, humid (19.97, 101519.8, 39.11524)
temp, baro, humid (19.97, 101523.3, 39.11621)
temp, baro, humid (19.98, 101514.4, 39.09277)
temp, baro, humid (19.99, 101517.9, 39.05957)
temp, baro, humid (20.0, 101517.0, 39.02539)
temp, baro, humid (19.99, 101513.5, 38.9209)
temp, baro, humid (20.0, 101519.6, 38.89746)
temp, baro, humid (20.0, 101514.3, 38.86328)
temp, baro, humid (20.01, 101521.3, 38.84082)
temp, baro, humid (20.02, 101519.5, 38.79492)
temp, baro, humid (20.02, 101511.6, 38.77148)
temp, baro, humid (20.03, 101515.9, 38.74903)

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports read

In [59]:
print("LTR-559 part_number/revision", i2c.readfrom_mem(0x23, 0x86, 1), "should be 9 2")
i2c.writeto_mem(0x23, 0x80, b'\x09')  # enable ALS gain x4 (light sensor)
for i in range(20):
    x = ustruct.unpack("h", i2c.readfrom_mem(0x23, 0x8A, 2))
    print(x)
    time.sleep_ms(500)

LTR-559 part_number/revision b'\x92' should be 9 2
(3112,)
(3106,)
(3100,)
(3123,)
(3489,)
(4414,)
(3875,)
(3855,)
(4156,)
(3956,)
(3914,)
(3947,)
(3876,)
(44,)
(48,)
(47,)
(47,)
(47,)
(47,)
(47,)


In [22]:
# Numbers all seem wrong coming out here (should attach 3.3V to the heater enable pin)

# start conversion | channel | gain(FSR) | single shot | 1600 SPS | disable comparator
channels = [0x4000, 0x5000, 0x6000, 0x7000]  # in0/gnd, in1, in2, in3 (fixed at 1.25V reference)
gainfsr, gainV = 0x0000, 6.144
for i in range(20):
    vals = [ ]
    for channel in channels:
        i2c.writeto_mem(0x49, 0x01, ustruct.pack(">H", 0x8000 | channel | gainfsr | 0x0100 | 0x0080 | 0x0003))
        time.sleep_ms(10)
        #for j in range(10000):
        #    x = i2c.readfrom_mem(0x49, 0x01, 2)
        #    x1 = ustruct.unpack(">H", x)[0]
        #    if (x1 & 0x8000):  # conversion complete
        #        break
        r = ustruct.unpack(">h", i2c.readfrom_mem(0x49, 0x00, 2))
        #vals.append(r[0]>>3)
        v = (r[0] >> 3)/2048.0*gainV
        vals.append(v)
    print("ADS1015", vals)
    #print("ADS1015", "ox", 56000*(3.3-vals[0]), "red", 56000*(3.3-vals[1]), "nh3", 56000*(3.3-vals[2]), "ref", vals[3])
    time.sleep_ms(500)

ADS1015 [2.634, 5.814, 4.206, 1.062]
ADS1015 [2.634, 5.814, 4.206, 1.044]
ADS1015 [2.634, 5.808, 4.206, 1.2]
ADS1015 [2.634, 5.814, 4.206, 1.158]
ADS1015 [2.634, 5.814, 4.206, 1.044]
.ADS1015 [2.634, 5.814, 4.206, 1.056]
ADS1015 [2.634, 5.814, 4.206, 1.218]
ADS1015 [2.634, 5.814, 4.206, 1.074]
ADS1015 [2.634, 5.808, 4.206, 1.026]
ADS1015 [2.634, 5.808, 4.206, 1.146]
ADS1015 [2.634, 5.814, 4.206, 2.178]
ADS1015 [2.622, 5.802, 4.188, 1.164]
ADS1015 [2.616, 5.802, 4.176, 1.314]
ADS1015 [2.598, 5.796, 4.164, 2.778]
ADS1015 [2.592, 5.796, 4.158, 1.152]
.ADS1015 [2.592, 5.79, 4.158, 1.062]
ADS1015 [2.58, 5.79, 4.146, 1.422]
ADS1015 [2.586, 5.796, 4.158, 2.064]
ADS1015 [2.592, 5.796, 4.164, 1.296]
ADS1015 [2.598, 5.796, 4.17, 1.026]


In [136]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [137]:
# Dust sensor is UART, connect enable to 3V
from machine import UART
u = UART(1, tx=4, rx=5, baudrate=9600)
print(u)

UART(1, baudrate=9600, bits=8, parity=None, stop=1, tx=4, rx=5, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=2)


In [139]:
import time, ustruct
from machine import UART
u = UART(1, tx=5, rx=4, baudrate=9600)
print(u)
cols = ["apm10", "apm25", "apm100", "pm10", "pm25", "pm100",
        "gt03um", "gt05um", "gt10um", "gt25um", "gt50um", "gt100um" ]
print("\t".join(cols))
for i in range(800):
    x = u.read()
    if x and len(x) == 32 and x[0] == 0x42 and x[1] == 0x4d:
        numbers = ustruct.unpack('>16H', x)
        csum = sum(x[:-2])
        if csum == numbers[-1]:
            assert(numbers[1] == 28) # frame length
            print("\t".join(map(str, numbers[2:-2])))
        else:
            print("bad checksum", x)
    time.sleep_ms(20)
        

UART(1, baudrate=9600, bits=8, parity=None, stop=1, tx=5, rx=4, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=2)
apm10	apm25	apm100	pm10	pm25	pm100	gt03um	gt05um	gt10um	gt25um	gt50um	gt100um
0	1	2	0	1	2	270	82	20	2	2	0
0	1	2	0	1	2	270	82	20	2	2	0
0	1	2	0	1	2	288	88	20	2	2	0
0	1	2	0	1	2	288	88	20	2	2	0
0	1	2	0	1	2	288	88	20	2	2	0
0	0	1	0	0	1	282	84	16	2	2	0
0	0	1	0	0	1	282	84	16	2	2	0
0	0	1	0	0	1	282	84	16	2	2	0
0	0	1	0	0	1	288	82	18	2	2	0
0	0	1	0	0	1	288	82	18	2	2	0
.0	0	1	0	0	1	288	82	18	2	2	0
0	0	0	0	0	0	288	82	18	0	0	0
0	0	0	0	0	0	288	82	18	0	0	0
0	0	0	0	0	0	288	82	18	0	0	0
0	0	0	0	0	0	264	76	16	0	0	0
0	0	0	0	0	0	264	76	16	0	0	0
0	0	0	0	0	0	264	76	16	0	0	0
0	0	0	0	0	0	282	80	18	0	0	0
0	1	1	0	1	1	282	80	18	0	0	0


In [68]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [125]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [126]:
# Settings for mapping the OLED bitmap into the RAM arrangement in its controller
width, height = 80, 160
ST7735_COLS, ST7735_ROWS = 132, 162
offset_left = (ST7735_COLS - width) // 2
offset_top = (ST7735_ROWS - height) // 2
x1 = width - 1
y1 = height - 1
y0 = offset_top
y1 += offset_top
x0 = offset_left
x1 += offset_left
print(0x2A, list(map(hex, bytearray([x0 >> 8, x0, x1 >> 8, x1]))))  # Column addr set
print(0x2B, list(map(hex, bytearray([y0 >> 8, y0, y1>>8, y1]))))    # Row addr set
#cmddata(0xB1, bytearray([0x02, 0x2C, 0x2D]))  # Frame rt 60Hz 333000/((1+20)*(160+0x2C+0x2D))



42 ['0x0', '0x1a', '0x0', '0x69']
43 ['0x0', '0x1', '0x0', '0xa0']


In [127]:

from machine import Pin, SPI

width, height = 80, 160

import framebuf, time
fdata = bytearray(width*height*2)
fbuf = framebuf.FrameBuffer(fdata, width, height, framebuf.RGB565)

spi = SPI(1, baudrate=10000000, polarity=0, phase=0, sck=Pin(14), mosi=Pin(13), miso=Pin(12))
cs = Pin(18, Pin.OUT)
dc = Pin(16, Pin.OUT)
backlight = Pin(17, Pin.OUT)


def color565(r, g, b):
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)

backlight.value(0)
time.sleep(0.1)
backlight.value(1)

def cmddata(cmd, data=None):
    cs.value(0)
    dc.value(0)
    spi.write(bytearray([cmd]))
    if data:
        dc.value(1)
        spi.write(data)
    cs.value(1)


In [128]:
print(333000/((2+20)*(160+0x2C+0x2D)))

60.78861


In [102]:
import time

# Initialize the display.
cmddata(0x01);  time.sleep_ms(120)     # Software reset
cmddata(0x11);  time.sleep_ms(120)     # Out of sleep mode
cmddata(0x13);  time.sleep_ms(120)     # Normal display mode
cmddata(0x29);  time.sleep_ms(120)     # Display on

#cmddata(0xB1, bytearray([0x02, 0x2C, 0x2D]))  # Frame rt 60Hz 333000/((1+20)*(160+0x2C+0x2D))
cmddata(0xC0, bytearray([0x02, 0x70]))     # 4.7V 1.0uA
#cmddata(0xC1, bytearray([0x05]))     # VGH 6X 14.7  VGL -3X -7.35
cmddata(0xC2, bytearray([0x01, 0x02]))     # Opamp current small, Boost frequency
cmddata(0xC3, bytearray([0x8A, 0x2A]))     # Power control
cmddata(0xC5, bytearray([0x3C, 0x38]))     # VCOMH 4.0, VCOML -1.050
cmddata(0x21)  # Don't invert display
cmddata(0x3A, bytearray([0x05]))     # 16-bit color
cmddata(0x36, bytearray([0x00]))     # Row order (0xC0 for reverse) + RGB

# gamma setting
cmddata(0xE0, bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29, 0x25, 0x2B, 0x39, 0x00, 0x01, 0x03, 0x10]))    # Set Gamma
cmddata(0xE1, bytearray([0x03, 0x1d, 0x07, 0x06, 0x2E, 0x2C, 0x29, 0x2D, 0x2E, 0x2E, 0x37, 0x3F, 0x00, 0x00, 0x02, 0x10]))    # Set Gamma

# Columns and rows embedding of 80x160 into 132x162
cmddata(0x2A, bytearray([0x00, 0x1A, 0x00, 0x69]))   # Column address 
cmddata(0x2B, bytearray([0x00, 0x01, 0x00, 0xA0]))   # Row address set



In [29]:
cmddata(0x20)  # Don't invert display


In [129]:
def display(fdata):
    cmddata(0x2C, fdata)       # write to RAM


In [130]:
fbuf.fill(0xFFFF)
for i in range(10):
    fbuf.fill_rect(10+i*5,i*10,8,8,0x0000)
    display(fdata)
    time.sleep(0.5)
fbuf.text("hi there", 10,100,0x0000)
display(fdata)


.

In [124]:
fbuf.fill(0x0000)
for x in range(0, width, 8):
    for y in range(0, height, 8):
        fbuf.fill_rect(x, y, 8, 8, 0xFFFF if (((x+y)//8)%2) == 0) else 0x0000)
display()



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fb47c6cf7d0, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [120]:
fbuf.fill(0x00e0)
fbuf.text("hi there", 10,120,0xffff)
display(fdata)

In [74]:
fbuf.fill(color565(0,0,255))
def color565(r, g, b):
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)
cmddata(0xE0, bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29, 0x25, 0x2B, 0x39, 0x00, 0x01, 0x03, 0x10]))    # Set Gamma
cmddata(0xE1, bytearray([0x03, 0x1d, 0x07, 0x06, 0x2E, 0x2C, 0x29, 0x2D, 0x2E, 0x2E, 0x37, 0x3F, 0x00, 0x00, 0x02, 0x10]))    # Set Gamma
fbuf.fill(0x6FFF)
display(fdata)

In [97]:
fbuf.fill(0x7EFF)
display(fdata)
print(list(fdata[:20]))

[255, 126, 255, 126, 255, 126, 255, 126, 255, 126, 255, 126, 255, 126, 255, 126, 255, 126, 255, 126]
